In [6]:
import pandas as pd
import numpy as np

target_cols = ['ind_cco_fin_ult1','ind_cder_fin_ult1','ind_cno_fin_ult1','ind_ctju_fin_ult1','ind_ctma_fin_ult1',
               'ind_ctop_fin_ult1','ind_ctpp_fin_ult1','ind_deco_fin_ult1','ind_deme_fin_ult1','ind_dela_fin_ult1',
               'ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1','ind_plan_fin_ult1','ind_pres_fin_ult1',
               'ind_reca_fin_ult1','ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1','ind_nomina_ult1',
               'ind_nom_pens_ult1','ind_recibo_ult1']

preds = np.load("preds1500.npy")
preds[preds < 0.09] = 0
print(preds[6:7,:])

#combined = zip(test_ids, preds)
target_cols = np.array(target_cols)
    
final_preds = [" ".join(list(target_cols[np.nonzero(pred)])) for pred in preds]    
test_ids = np.array(np.load("test_ids.npy"))

final_df = pd.DataFrame({'ncodpers':test_ids, 'added_products':final_preds})
final_df.to_csv("final_df.csv", index=False)

[[ 0.53950769  0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.11260043]]


In [12]:
import pandas as pd
import numpy as np
import xgboost as xgb

from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, KFold, StratifiedKFold

seed = 135

# load data
trainX = np.load("trainX.dat")
trainY = np.load("trainY.dat")
testX  = np.load("testX.npy")
print("train X is: ", trainX.shape)
print("train Y is: ", trainY.shape)
print("testX is: ", testX.shape)

X_train, X_test, y_train, y_test = train_test_split(trainX, trainY, test_size=0.2)

params = {'seed': 125,
          'learning_rate': 0.05,
          'max_depth':  5,
          'colsample_bytree': 0.9,
          'silent': 1,
          'subsample': 0.9,
          'objective': 'multi:softprob',
          'min_child_weight': 1,
          'eval_metric': 'mlogloss',
          'num_class' : 22
         }

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest  = xgb.DMatrix(X_test, label=y_test)

watchlist = [(dtrain, 'train'), (dtest, 'eval')]

bst = xgb.train(params, dtrain, 2500, watchlist, early_stopping_rounds=20, verbose_eval=50)

print("Booster best train score: {}".format(bst.best_score))
print("Booster best iteration: {}".format(bst.best_iteration))
print("Booster best number of trees limit: {}".format(bst.best_ntree_limit))


train X is:  (78713, 41)
train Y is:  (78713,)
testX is:  (929615, 41)
[0]	train-mlogloss:2.96112	eval-mlogloss:2.96162
Multiple eval metrics have been passed: 'eval-mlogloss' will be used for early stopping.

Will train until eval-mlogloss hasn't improved in 20 rounds.
[50]	train-mlogloss:1.7322	eval-mlogloss:1.76223
[100]	train-mlogloss:1.57411	eval-mlogloss:1.62745
[150]	train-mlogloss:1.52222	eval-mlogloss:1.59727
[200]	train-mlogloss:1.49443	eval-mlogloss:1.59072
Stopping. Best iteration:
[219]	train-mlogloss:1.48586	eval-mlogloss:1.59002

Booster best train score: 1.590022
Booster best iteration: 219
Booster best number of trees limit: 220
